# 🧠 Clasificación de Productos - Google Colab
Este notebook carga datos desde Google Drive, entrena una red neuronal convolucional (CNN) y evalúa su desempeño.

In [ ]:
# 📂 Montar Google Drive con dataset ya preprocesado
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 🚀 Clonar repositorio
!git clone https://github.com/solivare/DeepNeuralNetworkUSS.git
%cd DeepNeuralNetworkUSS/Examples/RetailProductClassification/

In [ ]:
# 📦 Instalar dependencias
!pip install -r requirements.txt

In [ ]:
# 📁 Ajustar paths para importar módulos
import os, sys
src_path = os.path.abspath('src')
if src_path not in sys.path:
    sys.path.append(src_path)

In [ ]:
# ⚙️ Copiar carpeta de Drive a entorno local (sólo imágenes preprocesadas)
!cp -r /content/drive/MyDrive/items_prepared ./data/items_prepared

## 🧠 Entrenamiento del Modelo

In [ ]:
from src import train, model, utils
import yaml

with open("config.yaml") as f:
    config = yaml.safe_load(f)

train_gen, val_gen = train.get_data_generators(config)
num_classes = len(train_gen.class_indices)

cnn_model = model.build_model(config, num_classes)
history = cnn_model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=config['training']['epochs']
)
utils.plot_training_history(history)

## 📊 Evaluación del Modelo

In [ ]:
from src import evaluate as ev
import numpy as np
from sklearn.metrics import classification_report

y_true = val_gen.classes
y_scores = cnn_model.predict(val_gen)
y_pred = np.argmax(y_scores, axis=1)
class_names = list(val_gen.class_indices.keys())

ev.plot_confusion_matrix(y_true, y_pred, class_names)
print(classification_report(y_true, y_pred, target_names=class_names))